In [ ]:
from sedona.spark import *
config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

In [ ]:
database = 'gde_gold'
sedona.sql(f'CREATE DATABASE IF NOT EXISTS org_catalog.{database}')

In [ ]:
sedona.sql(f'''
create or replace table org_catalog.{database}.clusters_gold as 
select 
a.sale_id,
a.geometry,
a.sale_price,
a.year_built,
a.sqft,
a.mean_ndvi_500m_radius,
b.avg_school_high_achievement,
c.pw_median_income_proxy,
c.est_total_pop
from org_catalog.gde_silver.house_sales_silver a
join org_catalog.gde_silver.homes_school_scores b using (sale_id)
join org_catalog.gde_silver.homes_demographics c using (sale_id)
''')

In [ ]:
sedona.sql(f''' 
create or replace table org_catalog.{database}.clusters_gold
SELECT
  geometry,
  sale_id,
  sale_price,
  year_built,
  sqft,
  mean_ndvi_500m_radius,
  avg_school_high_achievement,
  pw_median_income_proxy,
  est_total_pop,
  ST_DBSCAN(geometry, 100, 2, true) AS cluster
FROM org_catalog.{database}.clusters_gold
WHERE sale_price > 500000
  AND avg_school_high_achievement > 0.40;
''')

In [ ]:
sedona.sql(f'''select cluster.cluster from org_catalog.{database}.clusters_gold''').show()